In [3]:
#Scrape from website

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import mysql.connector
from mysql.connector import errorcode
from webdriver_manager.chrome import ChromeDriverManager
import time

# Database connection details
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Machengqi-4',
    'database': 'TBdatabase'
}

# Setup Chrome options
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Ensure GUI is off
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Setup Chrome service
service = Service(ChromeDriverManager().install())

# Initialize Chrome WebDriver with service and options
driver = webdriver.Chrome(service=service, options=options)

# Get the website
driver.get("https://www.savemyexams.com/ap/chemistry/college-board/22/topic-questions/")

# Allow some time for the page to load
time.sleep(5)

# Add explicit wait to ensure elements are loaded
wait = WebDriverWait(driver, 20)

# Initialize lists to store the scraped texts
topics = []
units = []

# Find all topics and their corresponding units
try:
    elements = driver.find_elements(By.CLASS_NAME, "d-block.link-body-emphasis")
    for element in elements:
        topic_text = element.text
        parent_div = element.find_element(By.XPATH, './/ancestor::div[@class="col-md-6"]')
        unit_element = parent_div.find_element(By.CLASS_NAME, "h6.mb-0")
        unit_text = unit_element.text
        topics.append(topic_text)
        units.append(unit_text)
except Exception as e:
    print("An error occurred while scraping:", e)
finally:
    driver.quit()

# Connect to the MySQL database
try:
    cnx = mysql.connector.connect(**db_config)
    cursor = cnx.cursor()

    # Create a new table ALChem
    create_table_query = """
    CREATE TABLE IF NOT EXISTS APChem (
        id INT AUTO_INCREMENT PRIMARY KEY,
        APChemTopics VARCHAR(255),
        Unit VARCHAR(255)
    )
    """
    cursor.execute(create_table_query)

    # Insert scraped data into the ALChem table
    insert_query = "INSERT INTO APChem (APChemTopics, Unit) VALUES (%s, %s)"
    for topic, unit in zip(topics, units):
        cursor.execute(insert_query, (topic, unit))

    # Commit the transaction
    cnx.commit()

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
finally:
    cursor.close()
    cnx.close()

# Print the scraped data
print("Scraped Data:")
for topic, unit in zip(topics, units):
    print(f"Topic: {topic} | Unit: {unit}")

Scraped Data:
Topic: 1.5 Atomic Structure & Electron Configuration | Unit: Unit 1: Atomic Structure & Properties
Topic: 2.3 Structure of Ionic Solids | Unit: Unit 2: Molecular & Ionic Compound Structure & Properties
Topic: 2.5 Lewis Diagrams | Unit: Unit 2: Molecular & Ionic Compound Structure & Properties
Topic: 3.4 Ideal Gas Law | Unit: Unit 3: Intermolecular Forces & Properties
Topic: 3.9 Separation of Solution & Mixtures Chromatography | Unit: Unit 3: Intermolecular Forces & Properties
Topic: 3.10 Solubility | Unit: Unit 3: Intermolecular Forces & Properties
Topic: 3.13 Beer-Lambert Law | Unit: Unit 3: Intermolecular Forces & Properties
Topic: 4.2 Net Ionic Equations | Unit: Unit 4: Chemical Reactions
Topic: 4.6 Introduction to Titration | Unit: Unit 4: Chemical Reactions
Topic: 5.1 Reaction Rates | Unit: Unit 5: Kinetics
Topic: 5.3 Concentration Changes Over TIme | Unit: Unit 5: Kinetics
Topic: 5.8 Reaction Mechanism & Rate Law | Unit: Unit 5: Kinetics
Topic: 6.4 Heat Capacity & C